In [ ]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Parkinsons_Disease"
cohort = "GSE80599"

# Input paths
in_trait_dir = "../../input/GEO/Parkinsons_Disease"
in_cohort_dir = "../../input/GEO/Parkinsons_Disease/GSE80599"

# Output paths
out_data_file = "../../output/preprocess/Parkinsons_Disease/GSE80599.csv"
out_gene_data_file = "../../output/preprocess/Parkinsons_Disease/gene_data/GSE80599.csv"
out_clinical_data_file = "../../output/preprocess/Parkinsons_Disease/clinical_data/GSE80599.csv"
json_path = "../../output/preprocess/Parkinsons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [ ]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [ ]:
import pandas as pd
import os
import json
from typing import Callable, Optional, Dict, Any
import numpy as np

# 1. Assessment of gene expression data availability
# Based on the background information, this dataset has gene expression data from Affymetrix Human Genome U219 platform
is_gene_available = True

# 2. Variable availability and data type conversion

# 2.1 Data Availability
# Trait (Parkinson's Disease progression) is at index 2 in the characteristics dictionary
trait_row = 2

# Age is at index 4 in the characteristics dictionary
age_row = 4

# Gender is at index 1 in the characteristics dictionary
gender_row = 1

# 2.2 Data Type Conversion Functions

def convert_trait(value):
    """Convert Parkinson's Disease progression to binary (0=Slow, 1=Rapid)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if "slow" in value.lower():
        return 0
    elif "rapid" in value.lower():
        return 1
    else:
        return None

def convert_age(value):
    """Convert age to a continuous numeric value"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Extract the numeric part (excluding "years" and other text)
    import re
    age_match = re.search(r'(\d+)', value)
    if age_match:
        age = int(age_match.group(1))
        # The age 8 might be a data entry error as this is an adult Parkinson's study
        if age < 18:  # Assuming this is an error
            return None
        return age
    return None

def convert_gender(value):
    """Convert gender to binary (0=Female, 1=Male)"""
    if pd.isna(value) or value is None:
        return None
    
    # Extract value after the colon if it exists
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    if value.lower() == 'female':
        return 0
    elif value.lower() == 'male':
        return 1
    else:
        return None

# 3. Save metadata
# Initial filtering on usability
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=trait_row is not None
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Load clinical data (assuming it's stored in a CSV or similar format)
    # The function assumes clinical_data is a DataFrame where each row is a feature type and columns are samples
    clinical_data_path = os.path.join(in_cohort_dir, "clinical_data.csv")
    
    # For demonstration, creating a simulated clinical_data DataFrame based on the characteristics dictionary
    # In a real scenario, you would load this from a file
    clinical_data = pd.DataFrame()
    for i, values in {0: ['tissue: whole blood'], 
                      1: ['gender: Male', 'gender: Female'], 
                      2: ["clinical classification: Rapid progression Parkinson's Disease patient", 
                         "clinical classification: Slow progression Parkinson's Disease patient"], 
                      3: ['updrs-mds3.12 score: 4', 'updrs-mds3.12 score: 3', 'updrs-mds3.12 score: 0', 
                         'updrs-mds3.12 score: 1', 'updrs-mds3.12 score: 2'], 
                      4: ['age at examination (years): 68', 'age at examination (years): 58', 
                         'age at examination (years): 53', 'age at examination (years): 54', 
                         'age at examination (years): 50', 'age at examination (years): 62']}.items():
        for j, value in enumerate(values):
            clinical_data.loc[i, f'Sample_{j+1}'] = value
    
    # Extract clinical features using the geo_select_clinical_features function
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical features
    preview = preview_df(selected_clinical_df)
    print("Preview of selected clinical features:", preview)
    
    # Create output directory if it doesn't exist
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    
    # Save the processed clinical data
    selected_clinical_df.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


### Step 3: Gene Data Extraction

In [ ]:
# 1. First get the file paths again to access the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the get_genetic_data function from the library to get the gene_data from the matrix_file
gene_data = get_genetic_data(matrix_file)

# 3. Print the first 20 row IDs (gene or probe identifiers) for future observation
print("First 20 gene/probe identifiers:")
print(gene_data.index[:20])


### Step 4: Gene Identifier Review

In [ ]:
# These appear to be Affymetrix microarray probe IDs (format: numeric_at or numeric_[x/s]_at)
# rather than standard human gene symbols like "SNCA" or "PARK7"
# These probe IDs will need to be mapped to their corresponding gene symbols

requires_gene_mapping = True


### Step 5: Gene Annotation

In [ ]:
# 1. First get the file paths using geo_get_relevant_filepaths function
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
gene_annotation = get_gene_annotation(soft_file)

# 3. Use the 'preview_df' function from the library to preview the data and print out the results.
print("Gene annotation preview:")
print(preview_df(gene_annotation))


### Step 6: Gene Identifier Mapping

In [ ]:
# 1. Determine which columns in gene_annotation contain probe IDs and gene symbols
# From the previous output, we can see:
# - 'ID' column contains probe IDs (like '11715100_at') which match our gene expression data
# - 'Gene Symbol' column contains the human gene symbols (like 'HIST1H3G')

# 2. Get the gene mapping dataframe
mapping_df = get_gene_mapping(gene_annotation, prob_col='ID', gene_col='Gene Symbol')
print("Gene mapping preview (first 5 rows):")
print(mapping_df.head())

# 3. Apply the gene mapping to convert probe-level data to gene-level data
gene_data = apply_gene_mapping(gene_data, mapping_df)
print(f"Shape of gene expression data after mapping: {gene_data.shape}")
print("First 5 gene symbols after mapping:")
print(gene_data.index[:5])

# Normalize gene symbols to ensure consistency and handle synonyms
gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Shape after normalization: {gene_data.shape}")
print("First 5 normalized gene symbols:")
print(gene_data.index[:5])

# Save the gene expression data
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data saved to {out_gene_data_file}")


### Step 7: Data Normalization and Linking

In [ ]:
# 1. Load the clinical data from the output file
clinical_data_path = out_clinical_data_file
clinical_df = pd.read_csv(clinical_data_path, index_col=0)
print(f"Loaded clinical data with shape: {clinical_df.shape}")
print("Clinical data preview:")
print(clinical_df.head())

# 2. Load the gene expression data from the output file
gene_data_path = out_gene_data_file
gene_df = pd.read_csv(gene_data_path, index_col=0)
print(f"Loaded gene expression data with shape: {gene_df.shape}")
print(f"First 5 gene symbols: {list(gene_df.index[:5])}")

# 3. Link the clinical and genetic data
# Need to examine the sample IDs in both clinical and gene data to ensure proper alignment
print(f"Clinical data column names (first 5): {list(clinical_df.columns[:5])}")
print(f"Gene data column names (first 5): {list(gene_df.columns[:5])}")

# Extract GSM IDs from the gene data columns - these should match our clinical samples
# GSM IDs in gene data columns are in format "GSM2129XXX"
gene_sample_ids = gene_df.columns.tolist()

# We need to extract real sample IDs from clinical data to match with gene data
# First check if clinical data has GSM IDs or needs transformation
if clinical_df.shape[1] > 0 and all(col.startswith('Sample_') for col in clinical_df.columns):
    print("Clinical data has generic sample IDs - need to map to actual GSM IDs")
    
    # Re-extract clinical data from the matrix file to get the GSM IDs
    _, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
    background_info, clinical_full = get_background_and_clinical_data(matrix_file)
    
    # Map sample positions to GSM IDs
    sample_id_map = {}
    for i, col in enumerate(clinical_full.columns):
        if col.startswith('!Sample_geo_accession'):
            continue
        if i-1 < len(clinical_df.columns):  # Skip the first column (GSM IDs)
            sample_id_map[clinical_df.columns[i-1]] = clinical_full.iloc[0, i]
    
    print(f"Sample ID mapping: {sample_id_map}")
    
    # Create new clinical DataFrame with GSM IDs
    clinical_gsm_df = pd.DataFrame(index=clinical_df.index)
    for sample_id, gsm_id in sample_id_map.items():
        if gsm_id in gene_df.columns:
            clinical_gsm_df[gsm_id] = clinical_df[sample_id]
    
    clinical_df = clinical_gsm_df
    print(f"Updated clinical data with GSM IDs, shape: {clinical_df.shape}")

# Filter gene data to only include columns that match clinical sample IDs
common_samples = [col for col in gene_df.columns if col in clinical_df.columns]
if not common_samples:
    print("No matching samples between clinical and gene data!")
    
    # Attempt to match using the first 6 samples of the gene data
    # (since we know we have 6 clinical samples)
    if clinical_df.shape[1] == 6 and gene_df.shape[1] >= 6:
        print("Attempting to match the first 6 samples...")
        clinical_df.columns = gene_df.columns[:6]
        common_samples = gene_df.columns[:6].tolist()

# After attempting to match samples, link the data
if common_samples:
    print(f"Found {len(common_samples)} common samples between clinical and gene data")
    filtered_gene_df = gene_df[common_samples]
    filtered_clinical_df = clinical_df[common_samples]
    
    # Link the data
    linked_data = pd.concat([filtered_clinical_df, filtered_gene_df], axis=0)
    linked_data = linked_data.T  # Transpose so samples are rows and features are columns
    
    print(f"Linked data shape: {linked_data.shape}")
    print("Linked data preview (first 5 columns, 5 rows):")
    print(linked_data.iloc[:5, :5])
    
    # 4. Handle missing values in the linked data
    trait_col = filtered_clinical_df.index[0]  # Use the first row from clinical data as trait
    print(f"Using '{trait_col}' as the trait column")
    
    # Check if we have sufficient trait data
    trait_data = linked_data[trait_col].dropna()
    print(f"Non-missing trait values: {len(trait_data)}/{len(linked_data)}")
    
    if len(trait_data) >= 10:  # Arbitrary threshold - we need enough samples with trait data
        handled_data = handle_missing_values(linked_data, trait_col=trait_col)
        print(f"Data shape after handling missing values: {handled_data.shape}")
        
        # 5. Determine whether the trait and demographic features are biased
        is_biased, handled_data = judge_and_remove_biased_features(handled_data, trait_col)
        
        # 6. Conduct final quality validation and save relevant information
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=len(trait_data) > 0,
            is_biased=is_biased,
            df=handled_data,
            note="This dataset contains gene expression profiles from peripheral blood of Parkinson's Disease patients with slow or rapid progression."
        )
        
        # 7. If the linked data is usable, save it
        if is_usable:
            os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
            handled_data.to_csv(out_data_file)
            print(f"Linked data saved to {out_data_file}")
        else:
            print("Dataset deemed not usable due to biased distribution. Data not saved.")
    else:
        print(f"Insufficient trait data ({len(trait_data)} samples). Dataset cannot be used.")
        
        # Record this information
        is_usable = validate_and_save_cohort_info(
            is_final=True,
            cohort=cohort,
            info_path=json_path,
            is_gene_available=True,
            is_trait_available=False,
            is_biased=None,
            df=linked_data,
            note="Insufficient trait data to perform analysis."
        )
        print("Dataset information recorded, but not saved due to insufficient trait data.")
else:
    print("Cannot proceed - no common samples between clinical and gene data")
    
    # Record this information
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=None,
        df=pd.DataFrame(),
        note="Unable to match clinical and gene expression samples."
    )
    print("Dataset information recorded, but not saved due to sample matching issues.")